# Model

Import necessary libraries.

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split

2024-04-24 08:53:11.148354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Read the augmented data.

In [2]:
lm = pd.read_json('new_data/lm_300wLP_anno_tr.json')

## Building the Model

Preparing the Data

In [69]:
# Create the X array.
X = []
for row in lm['landmarks_2d']:
    
    # Skip the row if it has less than 12 elements.
    if len(row) < 12:
        continue
    
    # Get the first 12 elements of the array.
    row = row[:12]
    
    # Flatten the array.
    row_ = np.array(row).reshape(12, 68 * 2)
    X.append(row_)

# Create the y array.
y = []
for row in lm['landmarks']:
    
    # Skip the row if it has less than 12 elements.
    if len(row) < 12:
        continue
    
    # Get only the first element.
    row = row[0]
    
    # Flatten the array.
    row_ = np.array(row).reshape(68 * 3)
    y.append(row_)

# Convert the lists to numpy arrays.
X = np.array(X)
y = np.array(y)

In [70]:
# Train-test split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

In [71]:
class LandmarkModel(models.Model):
    def __init__(self, in_, out_):
        super(LandmarkModel, self).__init__()
        self.lstm1 = layers.LSTM(64, return_sequences=True, input_shape=in_)
        self.lstm2 = layers.LSTM(64)
        self.dense = layers.Dense(out_, activation='linear')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        return self.dense(x)

In [72]:
# Get the input and output shapes.
in_ = X[0].shape
out_ = y[0].shape[0]

# Print the shapes.
print(f'{X[0].shape=}, {y[0].shape=}')

# Create the model.
model = LandmarkModel(in_, out_)
model.compile(optimizer='adam', loss='mse')

In [73]:
# Train the model.
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

X[0].shape=(12, 136), y[0].shape=(204,)
Epoch 1/50
75/75 [==============================] - 10s 40ms/step - loss: 46841.5156 - val_loss: 45688.6094
Epoch 2/50
75/75 [==============================] - 1s 14ms/step - loss: 44728.4570 - val_loss: 43753.4336
Epoch 3/50
75/75 [==============================] - 1s 14ms/step - loss: 42902.2969 - val_loss: 42009.5938
Epoch 4/50
75/75 [==============================] - 1s 13ms/step - loss: 41211.5234 - val_loss: 40368.2773
Epoch 5/50
75/75 [==============================] - 1s 12ms/step - loss: 39609.1719 - val_loss: 38803.5703
Epoch 6/50
75/75 [==============================] - 1s 13ms/step - loss: 38076.9336 - val_loss: 37303.4180
Epoch 7/50
75/75 [==============================] - 1s 13ms/step - loss: 36605.8438 - val_loss: 35860.9141
Epoch 8/50
75/75 [==============================] - 1s 13ms/step - loss: 35188.3750 - val_loss: 34469.4023
Epoch 9/50
75/75 [==============================] - 1s 13ms/step - loss: 33820.1484 - val_loss: 33126.0

In [ ]:
# Evaluate the model.
loss = model.evaluate(X_test, y_test)